# mmultの単体テスト

In [ ]:
import numpy as np
import cffi

from pynq import Overlay

# load Base Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()

import sys
sys.path.append("..")

from pynq_chainer import functions as F_
from pynq_chainer import links as L_
from pynq_chainer import overlays
from pynq_chainer import utils

In [ ]:
from pynq.drivers import xlnk

In [2]:
mmult = overlays.Mmult()
ffi = cffi.FFI()

In [3]:
def test(debug=False):
    w_size = (1024, 32)
    x_size = (512, 32)
    w_size = (1, 784)
    x_size = (32, 784)
    w_size = (4, 4)
    x_size = (4, 4)
    w = np.random.uniform(-1, 1, w_size).astype(np.float32)
    x = np.random.uniform(-1, 1, x_size).astype(np.float32)

    x_nrows, x_ncols = x.shape
    w_nrows, w_ncols = w.shape
    #y = np.zeros((w_nrows, x_nrows)).astype(np.float32)
    y, y_cdata = utils.malloc_cma_ndarray((w_nrows, x_nrows))
    
    x, x_cdata = utils.copy_cma_ndarray(x)
    w, w_cdata = utils.copy_cma_ndarray(w)

    #x_cdata = ffi.from_buffer(x.data)
    #w_cdata = ffi.from_buffer(w.data)
    #y_cdata = ffi.from_buffer(y.data)

    print('call')
    mmult(x_cdata, w_cdata, y_cdata, x_nrows, w_nrows, x_ncols)
    print('done')
    
    y_ = x.dot(w.T)
    y = y.T

    if debug:
        print("Actual(C):")
        print(y)
        print("Expected(NumPy):")
        print(y_)
    
    if np.allclose(y, y_, rtol=1e-04, atol=1e-04):
        print("OK")
    else:
        print("NG")

In [ ]:
for i in range(10):
    test(debug=True)

cma alloc
cma alloc
cma alloc
call
